In [ ]:
#from grid import *

In [4]:
config_file = "../code/grid.py"
with open(config_file) as f:
    code = compile(f.read(), config_file, 'exec')
    exec(code, globals(), locals())

In [5]:
#Continental
ant = Grid(res = [25*km, 25*km], left = -3100*km, up=3100*km, right = 3100*km, down = -3100*km)

#Local

#Global (almost)
world = Grid(crs_tgt=4326, res = [0.5, 0.5], left = -180, up=90, right = 180, down = -90)


We download a raster, in this case the global Natural Earth II with Shaded Relief. It is 100Mb.

In [ ]:
! mkdir -p ../../data/ne
! wget -nc https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/HYP_50M_SR_W.zip \
    -O ../../data/ne/HYP_50M_SR_W.zip
! unzip -n ../../data/ne/HYP_50M_SR_W.zip -d ../../data/ne

We import the raster to the global grid and to the Antarctic grid: 
The raster has three channels, so we assign it to X, Y and RGB coordinates. This might also take some time, as the raster needs to be warped to each grid. You can have your coffee now: 

In [ ]:
world.ds['RGB_RASTER_C'] = (('Y', 'X', 'RGB'), 
                world.read_raster('../../data/ne/HYP_50M_SR_W/HYP_50M_SR_W_C.tif'))

ant.ds['RGB_RASTER_C'] = (('Y', 'X', 'RGB'), 
                ant.read_raster('../../data/ne/HYP_50M_SR_W/HYP_50M_SR_W_C.tif'))

In [ ]:
ant.map_grid('RGB_RASTER_C')
world.map_grid('RGB_RASTER_C')

Pretty, but the raison d'etre for this code is to work properly in 3D. We download AN-1S sesimic 3D model of the Antarctic lithosphere. 

In [ ]:
! mkdir -p ../../data/an
! wget -nc http://www.seismolab.org/model/antarctica/lithosphere/AN1-S_depth_grd.tar.gz \
    -O ../../data/an/AN1-S_depth_grd.tar.gz
! tar -xvzf ./../data/an/AN1-S_depth_grd.tar.gz -C ../../data/an

In [ ]:
import glob

an_files = sorted(glob.glob('../data/An/AN1-S_depth_grd/*.grd'))
ant.ds.coords['AN_Z'] = [np.float32(d[-9:-4])*km for d in an_files]
str_depths = [str(d[-9:-4]) for d in an_files]

for d in ('S', 'Tc', 'Ts'):
    print('\n',d)
    an_array = np.empty((ant.nx, ant.ny, len(an_files)))
    an_array[:] = np.nan
    for i, a in enumerate(ant.ds.coords['AN_Z'].values):
        print(a//km, end=' ')
        fname = '../data/An/AN1-%s_depth_grd/AN1-%s_hslice_%s.grd'%(d,d,str_depths[i])
        if os.path.isfile(fname):
            an_array[:,:,i] = ant.read_grid(fname, xyz = ('x','y','z') ) 
    
    ant.ds['AN_%s'%d] = (('X', 'Y', 'AN_Z'), an_array)


In [ ]:
ant.ds['AN_S'].isel(AN_Z=30)

In [ ]:
ant.ds['AN_S'].sel(AN_Z=200*km)

In [ ]:
ant.grid_to_raster(ant.ds['AN_S'].sel(AN_Z=200*km), '../../An_S_200km.tiff')

In [ ]:
! mkdir -p data/raster
! wget -nc https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/GRAY_LR_SR_W.zip \
    -O data/raster/GRAY_LR_SR_W.zip
! unzip -n data/raster/GRAY_LR_SR_W.zip -d data/raster


## A detailed map

Now we try a high resolution dataset with a different projection. 

Again, we download some data, both raster and vector. Ref: [Norwegian Polar Institute 2014](https://data.npolar.no/dataset/19700322-beab-40e7-b867-c3f0c2756071)

Basemap is used to produce maps from the grid, it supports a [large range](https://matplotlib.org/basemap/users/mapsetup.html) of projections, but not all. [epsg 32731](https://epsg.io/32731) is not supported, but we can always use standard matplotlib or mayavi as the grid is regularly sampled. 

In [1]:
#Make object
bouvet = Grid(crs_tgt=32731, res = [5, 5], 
             up = 3974168, 
             left=517217 , 
             down = 3965208, 
             right = 529277)

print(bouvet.ds)

NameError: name 'Grid' is not defined

In [2]:
! mkdir -p ../../data/bouvet

! wget -nc https://publicdatasets.data.npolar.no/kartdata/NP_B0_DTM20.zip \
    -O ../../data/bouvet/NP_B0_DTM20.zip 
! unzip -n ../../data/bouvet/NP_B0_DTM20.zip -d ../../data/bouvet

! wget -nc https://publicdatasets.data.npolar.no/kartdata/NP_B20_SHP.zip \
    -O ../../data/bouvet/NP_B20_SHP.zip
! unzip -n ../../data/bouvet/NP_B20_SHP.zip -d ../../data/bouvet

! wget -nc https://content.satimagingcorp.com/static/galleryimages/landsat%208%20bouvet%20island%20lg.jpg \
    -O ../../data/bouvet/landsat_bouvet.jpg

File ‘../../data/bouvet/NP_B0_DTM20.zip’ already there; not retrieving.
Archive:  ../../data/bouvet/NP_B0_DTM20.zip
File ‘../../data/bouvet/NP_B20_SHP.zip’ already there; not retrieving.
Archive:  ../../data/bouvet/NP_B20_SHP.zip
File ‘../../data/bouvet/landsat_bouvet.jpg’ already there; not retrieving.


In [3]:
bouvet.ds['DEM'] = (('Y', 'X'), 
                bouvet.read_raster('../../data/bouvet/NP_B0_DTM20/B0_DTM20.tif'))



NameError: name 'bouvet' is not defined

In [4]:
bouvet.ds['LAND'] = (('Y', 'X'), 
                     bouvet.assign_shape('../../data/bouvet/NP_B20_SHP/B20_Land_f.shp', None, burn_val=1) )

NameError: name 'bouvet' is not defined

In [ ]:
plt.imshow(bouvet.ds['LAND'].values)
plt.show()

plt.imshow(bouvet.ds['DEM'].values)
plt.show()
bouvet.ds['DEM'][:,1222]